#Conversion de CFG a CNF

In [62]:
import copy
import re

In [ ]:
# Crear nuevo inicio y añadirlo a las entradas
# prueba = ['S->ABA','A->aA|ε','B->bBc|ε']

prueba = ['S->ASB|a', 'A->aAS|a|ε', 'B->SbS|A|bb']

nuevo_inicio = 'N->'+ prueba[0][0]

nueva_prueba = [None] * (len(prueba) + 1)

nueva_prueba[0] = nuevo_inicio

nueva_prueba[1:] = prueba

print(nueva_prueba)

In [ ]:
def addAndSymbol(input:list[str])->list[str]:
    operators = ['|','>','-','.']
    for regla in range(len(input)):
        reglas = list(input[regla])
        insertOffset = 0
        for i in range(1, len(input[regla])):
            if input[regla][i] not in operators and input[regla][i-1] not in operators:
                a = reglas[i]
                b = reglas[i-1]
                reglas.insert(i+insertOffset,'.')
                insertOffset += 1
        input[regla] = ''.join(reglas)
    return input

def stringListToCFG(input:list[str])->dict[str,set[str]]:
    input = addAndSymbol(input)
    input = [x.split('->') for x in input]
    input = [{x[0]:set(x[1].split('|'))} for x in input]
    return input
print(addAndSymbol(nueva_prueba))
print(stringListToCFG(nueva_prueba))

In [65]:
from itertools import combinations

# funciones de ayuda que producen las combinaciones de todas las letras que hay que eliminar
def remove_indices(string_,to_remove):
    # encontrar posición todo lo que necesita para eliminar
    positions_toremove =[]
    for j in range(len(string_)):
        if string_[j] == to_remove:
            positions_toremove.append(j)
    # generar combinaciones de este índice
    indecies=[]
    for j in range(1,len(positions_toremove)+1):
        indecies.extend(list(combinations(positions_toremove, r=j)))

    return indecies

In [66]:
# una función auxiliar para recorrer los índices y eliminarlos
def generate_combinations(string_,indecies):
    # bucle a través de la cadena y eliminar de acuerdo con la combinación
    temp_str=set()
    c=list(string_)
    for i in indecies:
        x=c.copy()
        for j in i:
            x[j]=''

        added = ''.join(x)
        if len(added) == 0:
            temp_str.add('ε')
        else:
            temp_str.add(added)

    return temp_str

In [67]:
# una función de ayuda para asegurarse de que no hay nulos para eliminar aún más
def no_null(CFG, start):
    for key, values in CFG.items():
        if "ε" in values and key != start:
            return False
    return True

In [68]:
# ========================== remove null productions
def remove_null_production(CFG, start):
    # terminado = []
    while no_null(CFG, start) == False:
        for key, values in CFG.items():
            # empezar por encontrar cualquier épsilon para poder eliminarlo y sustituirlo
            # si clave en acabado y "ε" en valores y clave != inicio:
            # eliminar el épsilon y darlo por terminado
            # CFG[key].remove('ε')

            if "ε" in values and key != start:
                # encontrar todos los valores de la clave
                CFG[key].remove('ε')
                for keys2, values2 in CFG.items():
                    # bucle para encontrar la llave que tenía la epsolina
                    for j in values2:
                        if key in ''.join(list(j)):
                            #buscar la clave en los valores
                            CFG[keys2] = CFG[keys2].union(generate_combinations(j,remove_indices(j,key)))
                #finished.append(key)
                CFGcopy = copy.deepcopy(CFG)
                for i in CFGcopy:
                    for j in CFGcopy[i]:
                        # noEps = re.search('[a-zA-Z]+', j)
                        CFG[i].add(j.replace('..', '.'))
                        if (j != j.replace('..', '.')):
                            CFG[i].remove(j)
                            j = j.replace('..', '.')
                        if (j[0] == '.' or j[-1] == '.'):
                            CFG[i].add(j.strip('.'))
                            CFG[i].remove(j)

    return CFG

In [ ]:
CFG = stringListToCFG(nueva_prueba)
CFG

In [ ]:
CFGdict = {}
for i in range (0, len(CFG)):
    CFGdict.update(CFG[i])

CFGdict

In [ ]:
CFG = remove_null_production(CFGdict, nuevo_inicio[0])
CFG

In [72]:
# una función de ayuda para buscar todos los nonterminales que están solos y los añade a una lista
def contain_nonterminal(set_value):
    set_of_nonterminals = set()
    for i in set_value:
        if len(i) == 1 and i.isupper():
            set_of_nonterminals.add(i)
    return set_of_nonterminals


In [73]:
# una función de ayuda para asegurarse de que ya no hay nontermianls solo
def done(CFG):
    for k, v in CFG.items():
        if len(contain_nonterminal(v)) != 0:
            return False
    return True

In [74]:
# =============================== Función final de producción unitaria
def remove_unit_production(CFG):

    while done(CFG) != True:
        # bucle a través del diccionario para encontrar nontemrminals que están solos
        for primary_key, set_value in CFG.items():
            set_non_terminals = contain_nonterminal(set_value)
            if len(set_non_terminals) != 0:
                #en caso de que haya varios nonterminales solos en un valor
                for i in set_non_terminals:
                    # suprimir todos los no terminales
                    CFG[primary_key].remove(i)
                    # # unión de los valores primarios y los valores no terminales buscados
                    CFG[primary_key] =  CFG[primary_key].union(CFG[i])
                    # eliminar si la clave primaria es igual a un no terminal
                    if i == primary_key and i in CFG[primary_key]:
                        CFG[primary_key].remove(i)
    return CFG

In [ ]:
#CFG2 = {"S": {"A.a", "B"}, "A": {"a", "b.c", "B"}, "B": {"A", "b.b"}}
CFG = remove_unit_production(CFG)
print(CFG)

In [76]:
# prueba = {'S': {'a.S', 'A', 'C'}, 'A': {'a'}, 'B': {'a.a'}, 'C': {'a.C.b'}}
# inicio = 'S'

def eliminateUselessProd(cfg, start):
    testCopy = copy.deepcopy(cfg)

    # no generador
    non_generating = set()
    for i in cfg[start]:
        for j in i:
            # caso1: si el estado inicial tiene un no-terminal que no es una clave, es decir, que no tiene transiciones, se añade al conjunto no generador.
            if (j not in cfg.keys() and j.isupper()):
                non_generating.add(j)
                continue
            # caso2: si un no terminal transiciona a un terminal Y no terminal, sin y épsilon, lo que significa que el no terminal no puede ser eliminado,
            # se considera no generador. Comprueba si está en mayúsculas y no tiene una transición épsilon y ninguna transición a terminal
            if ((j.isupper()) and ('' not in cfg[j]) and (len(cfg[j]) == 1)):
                for k in str(cfg[j]):
                    if k.isupper():
                        non_generating.add(j)

    # elimina el elemento que tiene un no-terminal no generador
    for i in testCopy[start]:
        for j in i:
            if ((j in non_generating)):
                cfg[start].remove(i)
    # borra el no-terminal no-generador del cfg
    for i in non_generating:
        del cfg[i]

    # No accesible
    reachable = {}
    non_reachable = {}

    for i in cfg[start]:
        for j in i:        # comprueba los no-terminales en el estado inicial y los añade al dict alcanzable con sus valores
            if (j.isupper()):
                reachable[j] = cfg[j]

    for i in cfg.keys():   # añade cualquier no-terminal encontrado en el CFG que no esté en el dict accesible y lo añade al dict no alcanzable
        if ((i not in reachable.keys()) and i != start):
            non_reachable[i] = cfg[i]

    for i in testCopy:     # recorre los no-terminales que no pueden ser alcanzados desde el estado inicial y los borra
        if i in non_reachable:
            del cfg[i]

    return cfg

CFG = eliminateUselessProd(CFG, nuevo_inicio[0])



In [ ]:
print(CFG)

In [ ]:
def ReduceStringToEvens(regla:str,originalInput:dict[str,set[str]],currentDict:dict[str,set[str]],currentCounter:int)->tuple[str,dict[str,str],int]:
    output = {}
    if regla.count('.') == 1:
        return regla,{},currentCounter
    splitString = regla.split('.')
    if(len(splitString)%2==1):
        evenRule = None
        firstTwoLiterals = regla[:regla.index('.',regla.index('.')+1)]
        if {firstTwoLiterals} in originalInput.values():
            newRuleKey = list(originalInput.keys())[list(originalInput.values()).index({firstTwoLiterals})]
            evenRule = regla.replace(firstTwoLiterals,newRuleKey)
        elif {firstTwoLiterals} not in currentDict.values():
            newRuleKey = f'K{currentCounter}'
            currentCounter += 1
            newRuleValue= firstTwoLiterals
            output[newRuleKey] = {newRuleValue}
            evenRule = regla.replace(firstTwoLiterals,newRuleKey)
        else:
            newRuleKey = list(currentDict.keys())[list(currentDict.values()).index({firstTwoLiterals})]
            evenRule = regla.replace(firstTwoLiterals,newRuleKey)
    else:
        evenRule = regla

    currentDict.update(output)
    if len(evenRule.split('.'))>2:
        left = '.'.join(splitString[:len(splitString)//2])
        right = '.'.join(splitString[len(splitString)//2:])
        leftSet = {left}
        if leftSet in originalInput.values():
            leftKey = list(originalInput.keys())[list(originalInput.values()).index(leftSet)]
        elif leftSet in output.values():
            leftKey = list(output.keys())[list(output.values()).index(leftSet)]
        elif leftSet not in currentDict.values():
            leftKey = currentCounter
            currentCounter += 1
            leftKey = f'K{leftKey}'
        else:
            leftKey = list(currentDict.keys())[list(currentDict.values()).index({left})]
        splitStringLeft = ReduceStringToEvens(left,originalInput,currentDict,currentCounter)
        currentCounter = splitStringLeft[2]
        output.update(splitStringLeft[1])
        output[f'{leftKey}'] = {splitStringLeft[0]}

        rightSet = {right}
        if rightSet in originalInput.values():
            rightKey = list(originalInput.keys())[list(originalInput.values()).index(rightSet)]
        elif rightSet in output.values():
            rightKey = list(output.keys())[list(output.values()).index(rightSet)]
        elif rightSet not in currentDict.values():
            rightKey = currentCounter
            currentCounter += 1
            rightKey = f'K{rightKey}'
        else:
            rightKey = list(currentDict.keys())[list(currentDict.values()).index(rightSet)]
        splitStringRight = ReduceStringToEvens(right,originalInput,currentDict,currentCounter)
        output[f'{rightKey}'] = {splitStringRight[0]}
        currentCounter = splitStringRight[2]
        output.update(splitStringRight[1])

        evenRule = f'{leftKey}.{rightKey}'

    return evenRule,output,currentCounter


def ToCNF(input : dict[str,set[str]])->dict[str,set[str]]:
    output = {}
    newRuleCounter = 0
    for k,v in input.items():
        FinalValue = set()
        for regla in v:
            if(len(regla)>1):
                elementos = regla.split('.')
                for i in range(len(elementos)):
                    if(elementos[i].islower()):
                        if {elementos[i]} in input.values():
                            newRuleKey = list(input.keys())[list(input.values()).index({elementos[i]})]
                            elementos[i] = newRuleKey
                        elif {elementos[i]} not in output.values():
                            newRuleKey = f'K{newRuleCounter}'
                            newRuleCounter += 1
                            newRuleValue = elementos[i]
                            output[newRuleKey] = {newRuleValue}
                            elementos[i] = newRuleKey
                        else:
                            newRuleKey = list(output.keys())[list(output.values()).index({elementos[i]})]
                            elementos[i] = newRuleKey
                regla = '.'.join(elementos)
            if len(regla.split('.'))<=2:
                FinalValue.add(regla)
                continue
            final = ReduceStringToEvens(regla,input,output,newRuleCounter)
            newRuleCounter = final[2]
            output.update(final[1])
            evenRule = final[0]
            FinalValue.add(evenRule)
        output[k] = FinalValue
    sortedOutput = {}
    for k in input.keys():
        sortedOutput[k] = output[k]
    sortedOutput.update(output)
    return sortedOutput

print(CFG)
ToCNF(CFG)